In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk



In [17]:
import pandas as pd
import re
from collections import Counter
import Helper_Aug_NLP as aug
import Extract_Data_From_Email as emailHelper

In [3]:
# Read security_settlement_emails.csv from Data folder
emailData = pd.read_csv('Data/security_settlement_emails.csv')
emailData.head()


,email_id,sender,receiver,subject,body,date
0,1,tradingdesk@bank.com,ops_team@bank.com,Request for SSI (Standard Settlement Instructi...,"Hi Team,\n\nThe trade ID T640196 has failed to...",2025-01-22 07:12:22
1,2,tradingdesk@bank.com,risk_mgmt@bank.com,Settlement Instruction - Equity Trade,"Dear Team,\n\nPlease confirm the settlement of...",2025-02-07 07:12:22
2,3,custodian@financegroup.com,compliance@bank.com,Trade Confirmation for ISIN XX1234567890,"Dear Compliance,\n\nWe have identified a discr...",2025-02-06 07:12:22
3,4,tradingdesk@bank.com,compliance@bank.com,Regulatory Compliance: Transaction Reporting,"Dear Risk Management,\n\nPlease ensure the rep...",2025-01-22 07:12:22
4,5,tradingdesk@bank.com,compliance@bank.com,Regulatory Compliance: Transaction Reporting,"Dear Risk Management,\n\nPlease ensure the rep...",2025-01-22 07:12:22


In [4]:
# Add the Subject in the email body as first line followed by the email body
emailData['body'] = emailData['subject'] + ' ' + emailData['body']
emailData.drop(columns=['subject'], inplace=True)
emailData.head()

,email_id,sender,receiver,body,date
0,1,tradingdesk@bank.com,ops_team@bank.com,Request for SSI (Standard Settlement Instructi...,2025-01-22 07:12:22
1,2,tradingdesk@bank.com,risk_mgmt@bank.com,Settlement Instruction - Equity Trade Dear Tea...,2025-02-07 07:12:22
2,3,custodian@financegroup.com,compliance@bank.com,Trade Confirmation for ISIN XX1234567890 Dear ...,2025-02-06 07:12:22
3,4,tradingdesk@bank.com,compliance@bank.com,Regulatory Compliance: Transaction Reporting D...,2025-01-22 07:12:22
4,5,tradingdesk@bank.com,compliance@bank.com,Regulatory Compliance: Transaction Reporting D...,2025-01-22 07:12:22


In [5]:
# Apply fumction aug.preprocess_text to clean the email body
emailData['cleaned_body'] = emailData['body'].apply(aug.preprocess_text)

In [6]:
# apply CountVectorizer to the cleaned email body
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
emailData['cleaned_body_str'] = emailData['cleaned_body'].apply(lambda x: ' '.join(x))
X = vectorizer.fit_transform(emailData['cleaned_body_str'])
X.shape


(1000, 66)

In [7]:
# Get word frequencies
word_freq = X.toarray().sum(axis=0)  # Sum word occurrences
top_words = sorted(zip(vectorizer.get_feature_names_out(), word_freq), key=lambda x: x[1], reverse=True)

# Display results
print("Top 10 most frequent words:")
for word, count in top_words:
    print(f"{word}: {count}")


Top 10 most frequent words:
settlement: 1542
trade: 1213
please: 842
attach: 495
action: 450
isin: 437
date: 433
operation: 432
instruction: 376
confirm: 347
good: 341
detail: 324
corporate: 279
resolution: 272
compliance: 268
report: 268
clearinghouse: 266
fail: 266
discrepancy: 260
regulatory: 255
custodian: 248
financegroup: 248
tradingdesk: 245
confirmation: 204
request: 194
counterparty: 175
investigate: 172
plan: 172
reason: 172
settle: 172
advise: 171
corrective: 171
identify: 171
review: 171
announce: 166
attention: 166
find: 166
ensure: 158
guideline: 158
management: 158
process: 158
reference: 158
reporting: 158
risk: 158
event: 113
mandatory: 113
notification: 113
equity: 109
ssi: 109
standard: 109
update: 109
collateral: 108
movement: 108
dispute: 100
mismatch: 100
transaction: 97
alert: 94
urgent: 94
query: 89
reconciliation: 89
amendment: 85
firm: 61
investment: 61
clearing: 58
house: 58
settlements: 37


In [8]:
# Most - Frequent Bigrams
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))
X2 = bigram_vectorizer.fit_transform(emailData['cleaned_body_str'])
X2.shape

# Get bigram frequencies
bigram_freq = X2.toarray().sum(axis=0)  # Sum bigram occurrences
top_bigrams = sorted(zip(bigram_vectorizer.get_feature_names_out(), bigram_freq), key=lambda x: x[1], reverse=True)

# Display results
print("Top 10 most frequent bigrams:")
for bigram, count in top_bigrams:
    print(f"{bigram}: {count}")

    

Top 10 most frequent bigrams:
settlement date: 433
settlement instruction: 376
settlement trade: 361
corporate action: 279
operation clearinghouse: 266
custodian financegroup: 248
trade please: 191
isin please: 177
confirm settlement: 175
date good: 175
isin counterparty: 175
please confirm: 175
trade isin: 175
confirm resolution: 172
fail settle: 172
investigate reason: 172
please investigate: 172
reason confirm: 172
resolution plan: 172
settle please: 172
trade fail: 172
advise corrective: 171
attach report: 171
compliance identify: 171
corrective action: 171
discrepancy settlement: 171
identify discrepancy: 171
please review: 171
report advise: 171
review attach: 171
action announce: 166
announce isin: 166
attach good: 166
attention operation: 166
detail attach: 166
find detail: 166
operation corporate: 166
please find: 166
attach detail: 158
date attach: 158
ensure reporting: 158
instruction trade: 158
management please: 158
please ensure: 158
process settlement: 158
reference sett

#### Start creating the prompt to classify if the intent type = Settelement(FC&IC)

In [12]:
delimiter = '#####'
email_text = ''

In [14]:
# Create prompt to find out if the Intent Type = 'Settlements (FC&IC)' or anything else

prompt = f"""
You are an intelligent business analysst in UBS. You work in operation department and you are responsible for the settlement of trades. You have received the below email from a client:
{email_text} 
{delimiter}
You need to determine if the email is related to 'Settlements (FC&IC)' or not.
You must extract Intent type from the email and return them in JSON format {{'IntentType': '<value>'}}. 
{delimiter}
Here are some instructions to help you determine the intent of the email:
Classify the email as:
1. **"Settlements (FC&IC)"** if the email discusses trade settlement, failed trades, reconciliation, clearing, or standard settlement instructions.
2. **"Unsure"** if the email is about other topics like general inquiries, compliance, or unrelated financial matters.
3. If you find the email contains information or tags related to SWIFT 5XX series, then you can be confident that the intent type is 'Settlements (FC&IC)'.
4. While reading the email content, try to check if the email discuss issues or inquiries related to security transfer or settlement enquiry between financial institutions or within a company? 
{delimiter}
Consider phrases like:{delimiter} 
'payment not received,' 'reconcile this transaction,' 'investigate this payment,' 'where is my payment,' 'incorrect payment amount,' 'requesting payment details,' 'confirming settlement,' 'discrepancy in settlement,' 
'reconciling accounts,' 'internal transfer,' 'intercompany billing','operation clearinghouse','custodian financegroup','isin counterparty','trade isin','instruction trade'.
5. You can also check if the email contains any of the following words: {delimiter}
'settlement', 'trade', 'isin', 'date', 'operation', 'instruction','clearinghouse','swift'.
6. If you are confident that the email is related to 'Settlements (FC&IC)', then you can send the response as {{'IntentType': 'Settlements (FC&IC)'}}.
7. If you are still unsure, send 'NA' as the intent type in JSON format like this: {{'IntentType': 'NA'}}.
{delimiter}
Here are some examples of the Intent Type:
Example 1
Input: {delimiter}
Dear Team,

Please confirm the settlement of trade ID T725375 for ISIN XX2305658993 with counterparty Clearing House C. Settlement date: 2025-02-08.

Best regards,
operations@clearinghouse.com
{delimiter}
Output: {delimiter}
{{'IntentType': 'Settlements (FC&IC)'}}
{delimiter}
Example 2:
Input: {delimiter}
Hello 

Please provide market reference and settlement reference for the below trade.

MG5GTNLX00

Thanks
Jack Rol
Security Services AG
{delimiter}
Output: {delimiter}
{{'IntentType': 'Settlements (FC&IC)'}}
{delimiter}
Example 3:
Input: {delimiter}
Hello

Do you have an update on the mentioned trade?

{{1:F01YOURBICXXX}}{{2:I548YOURBICXXX}}{{3:{{108:YOURREF}}}}{{4:
:20C::SETR//YOURREF
:22A::TRAD//BUY
:94B::AGRE//YOUR AGREEMENT DETAILS
:98A::PREA//20240726
:98A::TRAD//20240724
:98A::SETT//20240728
:35B::ISIN XYZ123456789//XYZ CORP/100/SHARES
:36B::QTTY//100
:11A::CURC//USD
:19A::MONE//10000.00
:70::TRDT//Trade details - XYZ Corp Shares
:25D::ACCT//YOUR_SAFEKEEPING_ACCOUNT_NUMBER
:25D::PSET//YOUR_SETTLEMENT_ACCOUNT_NUMBER
:82A::RECE//COUNTERPARTY_BICXXX
:86A::DELI//YOURBICXXX
:97A::SAFE//YOUR_SAFEKEEPING_ACCOUNT_NUMBER
:90A::TRAD//PRICE//100.00
-}}{{5:CHK:CHECKSUM}}

Regards
John
{delimiter}
Output: {delimiter}
{{'IntentType': 'Settlements (FC&IC)'}}
{delimiter}
Example 4:
Input: {delimiter}
Hello Team

We are expecting a payment from your side. Please confirm the payment status.

Thanks
John
{delimiter}
Output: {delimiter}
{{'IntentType': 'NA'}}
"""

In [15]:
# Add the prompt to the messages
messages=[{"role": "system", "content": prompt}]

In [19]:
# Read the email
with open('Data\Settlement_FI_IC_Email.txt') as f:
    email_text = f.readlines()

# Convert the email to string
email_text = ''.join(email_text)
email_text = email_text.lower()

In [20]:
intentType = emailHelper.callChatCompletion(messages,email_text)
print(intentType)

{'IntentType': 'Settlements (FC&IC)'}
